# Saving the Models after Training

In [8]:
import numpy as np
import torch
import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Setting device to CUDA (optional)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8,out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.fc1 = nn.Linear(16*7*7, num_classes)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        
        return x

def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("Saving Checkpoint ==> ")
    torch.save(state, filename)
    
def load_checkpoint(checkpoint):
    print("Loading Checkpoint ==>")
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    # IF YOU HAVE SAVED MORE THINGS LIKE ACCURACY SO KINDLY LOAD THOSE ITEMS FROM DICTIONARY

    
# model = CNN()
# x = torch.randn(299, 1, 28, 28) # 299 examples, 1 channel, 28 x 28 size
# print(model(x).shape)

# Hyperparameters setting
in_channels = 1
num_classes = 10
learning_rate = 1e-4
batch_size = 1024
num_epochs = 6
load_model = True

# Load the Data Train & Test
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)



# Initialize the Network
model = CNN().to(device)

# LOSS & OPTIMIZER
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

if load_model:
    load_checkpoint(torch.load("my_checkpoint.pth.tar"))

# Train the Network
for epoch in tqdm.tqdm(range(num_epochs)):
    losses = [] # FOR STORING THE LOSSES
    
    if epoch%2 == 0:
        checkpoint = {'state_dict' : model.state_dict(), 'optimizer' : optimizer.state_dict()} # YOU CAN STORE OTHERS LIKE ACC ALSO
        save_checkpoint(checkpoint)
    
    for batch_idx, (data, targets) in enumerate(train_loader):
        
        # Convert the data to the CUDA for faster training
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # Flattening the data
        # data = data.reshape(data.shape[0], -1)
        
        # Forward
        scores = model(data)
        loss = criterion(scores, targets)
        losses.append(loss.item()) # APPENDING THE LOSS
        
        # Backward
        optimizer.zero_grad() # so that they dont store the gradrients
        loss.backward() # gradients computed here
        
        # gradient steps 
        optimizer.step() # here we update the weights based on the gradients computed on top loss.backward()
        
    mean_loss = sum(losses)/len(losses)
    print("Loss at epoch {} is {}".format(epoch, np.round(mean_loss, 2)))
        
        
# Check Accuracy on training & test set 

def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on Train Data!!... ")
    else:
        print(" Checking accuracy on Test Data!!  ")
    
    num_correct = 0
    num_samples = 0
    model.eval() # let the model know that it is in evaluation mode
    
    # to let the model know that you dont have to compute the gradients while doing the tesing/accuracy checking
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device)
            y = y.to(device)
            
#             x = x.reshape(x.shape[0], -1) # Flattening
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
        # print('{}/{} with accuracy {}'.format(num_correct, num_samples, np.round(num_correct/num_samples, 2)))
        
    model.train() 
    return num_correct/num_samples
    
print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}") 
            


  0%|                                                                                                                                                                                                                 | 0/6 [00:00<?, ?it/s]

Loading Checkpoint ==>
Saving Checkpoint ==> 


 17%|█████████████████████████████████▌                                                                                                                                                                       | 1/6 [00:04<00:22,  4.49s/it]

Loss at epoch 0 is 2.0


 33%|███████████████████████████████████████████████████████████████████                                                                                                                                      | 2/6 [00:08<00:17,  4.46s/it]

...

 83%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                 | 5/6 [00:22<00:04,  4.43s/it]

Loss at epoch 4 is 0.81


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:26<00:00,  4.45s/it]

Loss at epoch 5 is 0.67
Checking accuracy on Train Data!!... 


Accuracy on training set: 84.281662
 Checking accuracy on Test Data!!  
Accuracy on test set: 85.57
